## **Install Required Python Libraries**

You are editing this notebook, so we assume you have already cloned the repository from  
[https://github.com/klinucsd/interagency-tracking-system](https://github.com/klinucsd/interagency-tracking-system) into the folder ~/work/interagency-tracking-system. Please verify that the `requirements.txt` file is available in that folder before proceeding.

In [1]:
! pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


## **Download CNRA data in zip format**

In [4]:
from utils.download_from_box import download_box_shared_link

cnra_box_link = ""
download_box_shared_link(cnra_box_link)

Accessing shared link...
Found 1 file(s)

[1/1] Downloading: CNRA_TRMT_DATA_Pv6c_20251017.zip
    ✓ Saved: CNRA_TRMT_DATA_Pv6c_20251017.zip (59,301,413 bytes)



['./CNRA_TRMT_DATA_Pv6c_20251017.zip']

## **Unzip CNRA Data**

In [6]:
! unzip CNRA_TRMT_DATA_Pv6c_20251017.zip

Archive:  CNRA_TRMT_DATA_Pv6c_20251017.zip
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000001.gdbindexes  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000001.gdbtable  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000001.gdbtablx  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000001.TablesByName.atx  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000002.gdbtable  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000002.gdbtablx  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000003.gdbindexes  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000003.gdbtable  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000003.gdbtablx  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000004.CatItemsByPhysicalName.atx  
  inflating: _ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb/a00000004.CatItemsByType.atx  
  inflating: _ags_data0A9553925C34457

## **Unzip Reference Data in GDB format**

In [2]:
! unzip a_Reference.gdb.zip 

Archive:  a_Reference.gdb.zip
   creating: a_Reference.gdb/
  inflating: a_Reference.gdb/a00000007.CatRelTypesByBackwardLabel.atx  
  inflating: a_Reference.gdb/a0000000c.horizon  
  inflating: a_Reference.gdb/a0000000c.gdbtablx  
  inflating: a_Reference.gdb/a00000007.gdbtablx  
  inflating: a_Reference.gdb/timestamps  
  inflating: a_Reference.gdb/a00000010.spx  
  inflating: a_Reference.gdb/a00000011.I36Original_Acti.atx  
  inflating: a_Reference.gdb/a00000006.FDO_UUID.atx  
  inflating: a_Reference.gdb/a00000007.gdbtable  
  inflating: a_Reference.gdb/a00000004.gdbtablx  
  inflating: a_Reference.gdb/a00000010.freelist  
  inflating: a_Reference.gdb/a0000000b.gdbtablx  
  inflating: a_Reference.gdb/a00000001.gdbindexes  
  inflating: a_Reference.gdb/a00000009.horizon  
  inflating: a_Reference.gdb/a0000000b.gdbtable  
  inflating: a_Reference.gdb/a00000011.gdbtable  
  inflating: a_Reference.gdb/a0000000a.horizon  
  inflating: a_Reference.gdb/a00000003.gdbtable  
  inflating: a_R

## **Enriching CNRA Data**

Converts the California Department of Natural Resources's fuels treatments dataset into the Task Force standardized schema.  Dataset is enriched with vegetation, ownership, county, WUI, Task Force Region, and year.             

In [ ]:

import warnings
import logging
import time
from datetime import datetime
import psutil
import os
from enrich.enrich_CNRA import enrich_CNRA

# Get the current process ID
process = psutil.Process(os.getpid())

cnra_input_gdb_path = "_ags_data0A9553925C34457A8C2BB35A44FA88B7.gdb"
cnra_polygon_layer_name = "TREATMENT_POLY"
cnra_line_layer_name = "TREATMENT_LINE"
cnra_point_layer_name = ""
cnra_project_polygon_layer_name = ""
cnra_activity_layer_name = ""

a_reference_gdb_path = "a_Reference.gdb"
start_year, end_year = 2023, 2025

output_format_dict = {'start_year': start_year,
                      'end_year': end_year,
                      'date': datetime.today().strftime('%Y%m%d')}
output_gdb_path = f"/tmp/CNRA_{start_year}_{end_year}.gdb"
output_layer_name = f"IFPRS_enriched_{datetime.today().strftime('%Y%m%d')}"

enrich_CNRA(cnra_input_gdb_path,
            cnra_polygon_layer_name,
            cnra_line_layer_name,
            cnra_point_layer_name,
            cnra_project_polygon_layer_name,
            cnra_activity_layer_name,
            a_reference_gdb_path,
            start_year,
            end_year,
            output_gdb_path,
            output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")